In [107]:
import os
from datetime import datetime as dt
from random import randint
from collections import defaultdict
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('dark_background')

import torchaudio
import torch
import torch.nn as nn
import torch.nn.functional as F
from tensorboardX import SummaryWriter

from hparams import Hparam
from data.dataset import SpeechDataset
from cpc_model import CPCModel

In [101]:
config = Hparam('./cpc_config.yaml')

gettime = lambda: str(dt.time(dt.now()))[:8]

## workbench

## model

In [25]:
ds = SpeechDataset(config.data.path)

100%|██████████| 28539/28539 [02:00<00:00, 237.25it/s]


In [77]:
model = CPCModel(config).to(config.train.device)

# Model summary

In [78]:
from torchsummaryX import summary

In [79]:
t = ds[1][1].unsqueeze(0).unsqueeze(0)
f = summary(model, t.cuda())

                                     Kernel Shape    Output Shape     Params  \
Layer                                                                          
0_convolutions.Conv1d_0              [1, 512, 10]  [1, 512, 4095]     5.632k   
1_convolutions.ReLU_1                           -  [1, 512, 4095]          -   
2_convolutions.BatchNorm1d_2                [512]  [1, 512, 4095]     1.024k   
3_convolutions.Conv1d_3             [512, 512, 8]  [1, 512, 1023]  2.097664M   
4_convolutions.ReLU_4                           -  [1, 512, 1023]          -   
5_convolutions.BatchNorm1d_5                [512]  [1, 512, 1023]     1.024k   
6_convolutions.Conv1d_6             [512, 512, 4]   [1, 512, 512]  1.049088M   
7_convolutions.ReLU_7                           -   [1, 512, 512]          -   
8_convolutions.BatchNorm1d_8                [512]   [1, 512, 512]     1.024k   
9_convolutions.Conv1d_9             [512, 512, 4]   [1, 512, 257]  1.049088M   
10_convolutions.ReLU_10                 

# training

In [1]:
import os

In [ ]:
lds = torchaudio.datasets.LIBRISPEECH('/data/deepvk/librispeech/train-clean-100/', url='train-clean-100')